# Text-to-SQL Workflow

In this example, we adapt the [code](https://github.com/ShayanTalaei/CHESS/tree/fc6f0b7ef34ccb573d764be8fba52b4afdd20ff5) from the paper [CHESS: Contextual Harnessing for Efficient SQL Synthesis](https://arxiv.org/abs/2405.16755).  

The workflow is as follows:

![CHESS](../imgs/CHESS_workflow.png)

First, run the pre-processing script in `./run/run_preprocess.sh` to create the databases. This should generate a `data` folder. Ensure your `.env` file contains the following keys:
- `OPENAI_API_KEY`
- `DB_ROOT_PATH`, which should be set to the path of `data/dev`

## Data loader

The original repository expects command line arguments passed into its data-loader. We can preserve the original parser function and just set the arguments in the script itself. This dataset does not contain a ground truth, so we pass in an empty dictionary `{}` as the second value in the tuple.

Then, we use the data files generated by the pre-processing step in the `data` folder.

In [12]:
import json
import cognify 
from src.utils import parse_arguments

import cognify
import numpy as np

import dotenv
dotenv.load_dotenv()

@cognify.register_data_loader
def load_data():
    args = parse_arguments()

    def read_from_file(data_path, args):
        with open(data_path, "r") as file:
            dataset = json.load(file)

        inputs = []
        for data in dataset:
            inputs.append(
                {
                    'args': args,
                    'dataset': [data],
                }
            )
        eval_data = [(input, {}) for input in inputs] # no ground truth in this case, set to empty dictionary
        return eval_data

    all_train = read_from_file('/mnt/ssd4/lm_compiler/examples/CHESS/data/dev/other_sub_sampled.json', args)
    test_set = read_from_file('/mnt/ssd4/lm_compiler/examples/CHESS/data/dev/sub_sampled_bird_dev_set.json', args)
    
    # shuffle the data
    all_train = np.random.permutation(all_train).tolist()
    return all_train[:100], all_train[100:], test_set[:10]

## Evaluator

In this repository, they run the SQL code in a sandbox environment and return the result. Hence, our evaluator does not need to re-execute the code. Instead, it can just return whether the result was correct.  

In [9]:
@cognify.register_evaluator
def eval_text_to_sql(counts):
    """
    Evaluate the statistics of the run.
    """
    correct = any(vs['correct'] == 1 for vs in counts.values())
    return correct

## Optimizer

Let's select the text-to-sql search from the CogHub. This search function automatically applies chain-of-thought, planning, and few-shot examples wherever it is best.

In [10]:
from cognify.hub.search import text_to_sql
search_settings = text_to_sql.create_search()

## Start the optimization

We've provided the 3 code blocks above in `src/configy.py`. With the Cognify CLI, you can start the optimization like this:

```console
$ cognify optimize /full/path/to/src/workflow.py
```

Alternatively, you can run the following cell (*warning*: this workflow may run for quite some time):

In [11]:
train, val, dev = load_data()

opt_cost, pareto_frontier, opt_logs = cognify.optimize(
    script_path="/mnt/ssd4/reyna/cognify/examples/CHESS/src/workflow.py",
    control_param=search_settings,
    train_set=train,
    val_set=val,
    eval_fn=eval_text_to_sql,
    force=True, # This will overwrite the existing results
)

Namespace(data_mode='dev', pipeline_nodes='keyword_extraction+entity_retrieval+context_retrieval+column_filtering+table_selection+column_selection+candidate_generation+revision+evaluation', pipeline_setup='{\n    "keyword_extraction": {\n        "engine": "gpt-4o-mini",\n        "temperature": 0.2,\n        "base_uri": ""\n    },\n    "entity_retrieval": {\n        "mode": "ask_model"\n    },\n    "context_retrieval": {\n        "mode": "vector_db",\n        "top_k": 5\n    },\n    "column_filtering": {\n        "engine": "gpt-4o-mini",\n        "temperature": 0.0,\n        "base_uri": ""\n    },\n    "table_selection": {\n        "mode": "ask_model",\n        "engine": "gpt-4o-mini",\n        "temperature": 0.0,\n        "base_uri": "",\n        "sampling_count": 1\n    },\n    "column_selection": {\n        "mode": "ask_model",\n        "engine": "gpt-4o-mini",\n        "temperature": 0.0,\n        "base_uri": "",\n        "sampling_count": 1\n    },\n    "candidate_generation": {\n 

> Evaluation in dry_run | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0/100 [00:00<?, ?it/s]

Result directory: results/european_football_2/1101/2024-12-03-16-00-57
Total number of tasks: 1
Result directory: results/student_club/1331/2024-12-03-16-00-57
Total number of tasks: 1
Result directory: results/codebase_community/558/2024-12-03-16-00-57
Result directory: results/superhero/769/2024-12-03-16-00-57Total number of tasks: 1

Total number of tasks: 1Result directory: results/codebase_community/685/2024-12-03-16-00-57

Total number of tasks: 1
Result directory: results/codebase_community/572/2024-12-03-16-00-57
Total number of tasks: 1
Result directory: results/formula_1/1010/2024-12-03-16-00-57Result directory: results/formula_1/866/2024-12-03-16-00-57

Total number of tasks: 1Total number of tasks: 1

Result directory: results/debit_card_specializing/1509/2024-12-03-16-00-57
Total number of tasks: 1
Result directory: results/formula_1/932/2024-12-03-16-00-57
Total number of tasks: 1
Result directory: results/european_football_2/1060/2024-12-03-16-00-57
Total number of tasks

2024-12-03 16:01:33,366 - /mnt/ssd4/reyna/cognify/examples/CHESS/src/runner/logger.py:79 - WARNING: (superhero, 769) Missed column: 'superhero'.'id' not in selected_columns
2024-12-03 16:01:33,367 - /mnt/ssd4/reyna/cognify/examples/CHESS/src/runner/logger.py:79 - WARNING: (superhero, 769) Missed column: 'attribute'.'id' not in selected_columns
2024-12-03 16:01:33,368 - /mnt/ssd4/reyna/cognify/examples/CHESS/src/runner/logger.py:79 - WARNING: (superhero, 769) Missed column: 'publisher'.'id' not in selected_columns


Result directory: results/student_club/1357/2024-12-03-16-01-35
Total number of tasks: 1


...

2024-12-03 16:07:21,026 - /mnt/ssd4/reyna/cognify/examples/CHESS/src/database_utils/execution.py:119 - ERROR: Error in validate_sql_query: no such column: name


Total number of tasks: 1

2024-12-03 16:07:21,027 - /mnt/ssd4/reyna/cognify/examples/CHESS/src/database_utils/execution.py:153 - WARNING: No valid SQL clusters found. Returning the first SQL query.


2024-12-03 16:07:21,060 - /mnt/ssd4/reyna/cognify/examples/CHESS/src/database_utils/execution.py:50 - ERROR: Error in execute_sql: no such column: name
SQL: SELECT AVG(milliseconds) FROM pitStops WHERE driverId = (SELECT driverId FROM drivers WHERE name = 'Lewis Hamilton'); 
2024-12-03 16:07:21,062 - /mnt/ssd4/reyna/cognify/examples/CHESS/src/database_utils/execution.py:119 - ERROR: Error in validate_sql_query: no such column: name
2024-12-03 16:07:21,912 - /mnt/ssd4/reyna/cognify/examples/CHESS/src/runner/logger.py:79 - WARNING: (thrombosis_prediction, 1235) Missed column: 'patient'.'diagnosis' not in selected_columns
2024-12-03 16:07:21,917 - /mnt/ssd4/reyna/cognify/examples/CHESS/src/database_utils/execution.py:50 - ERROR: Error in execute_sql: no such column: name
SQL: SELECT AVG(milliseconds) FROM pitStops WHERE driverId = (SELECT driverId FROM drivers WHERE name = 'Lewis Hamilton'); 
2024-12-03 16:07:21,918 - /mnt/ssd4/reyna/cognify/examples/CHESS/src/database_utils/execution.py:

KeyboardInterrupt: 